In [ ]:
# ab combined 后预测MusMusculus

import pandas as pd

data=pd.read_csv('./vdjdb.txt',sep='\t', header=0)
data[data['species'] == 'MusMusculus']
data=data[data['vdjdb.score'] != 0]
data

In [ ]:
columns_to_drop = ['antigen.species','antigen.gene','reference.id', 'method', 'meta','cdr3fix','web.method','web.method.seq','web.cdr3fix.nc','web.cdr3fix.unmp','mhc.a','mhc.b','mhc.class']
data.drop(columns=columns_to_drop, inplace=True)
df=data.dropna()
# # Filter data for alpha chain and beta chain
data_alpha = df[df['gene'] == 'TRA'].copy()
data_beta = df[df['gene'] == 'TRB'].copy()

data_alpha = data_alpha[data_alpha['complex.id'] != 0]
data_beta = data_beta[data_beta['complex.id'] != 0]

data_combined = pd.merge(data_alpha,data_beta, on='complex.id' )
data_beta.to_csv('mouse_combined.csv')
data_combined

In [ ]:
class_counts = data_combined['antigen.epitope_x'].value_counts()

# Find classes that have only one instance
single_classes = class_counts[class_counts == 1].index
data_combined = data_combined[~data_combined['antigen.epitope_x'].isin(single_classes)]

# 首先，根据`cdr3_a_aa`、`v_a_gene`和`j_a_gene`来计算距离矩阵
# 计算距离矩阵的具体代码将根据所选算法和数据类型有所不同

# 定义特征列和目标列
feature_columns = ['cdr3_x', 'cdr3_y', 'v.segm_x', 'j.segm_x', 'v.segm_y', 'j.segm_y' ]
target_column = 'antigen.epitope_x'

# 提取特征和目标
X = data_combined[feature_columns]  # 或是距离矩阵，取决于您的模型需求
y = data_combined[target_column]